<a href="https://colab.research.google.com/github/ggentile/TCC_beauty/blob/main/01_analise_exploratoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nova seção

In [1]:
from google.colab import drive

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!git config --global user.name "Gianni Gentile"

In [8]:
!git config --global user.email "giannikleiss4@gmail.com"

In [6]:
!git clone https://github.com/ggentile/TCC_beauty.git

Cloning into 'TCC_beauty'...
remote: Enumerating objects: 44, done.
remote: Total 44 (delta 0), reused 0 (delta 0), pack-reused 44 (from 2)
Receiving objects: 100% (44/44), 84.98 MiB | 31.83 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [7]:
%cd TCC_beauty/


/content/TCC_beauty


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')

In [8]:
orders_raw = pd.read_csv('/content/TCC_beauty/tcc-beleza/data/raw/models/olist_orders_dataset.csv')
order_items_raw = pd.read_csv('/content/TCC_beauty/tcc-beleza/data/raw/models/olist_order_items_dataset.csv')
order_payments_raw = pd.read_csv('/content/TCC_beauty/tcc-beleza/data/raw/models/olist_order_payments_dataset.csv')
products_raw = pd.read_csv('/content/TCC_beauty/tcc-beleza/data/raw/models/olist_products_dataset.csv')

In [29]:
print('Columns in orders_raw:', orders_raw.columns.tolist())
print('Columns in order_items_raw:', order_items_raw.columns.tolist())
print('Columns in order_payments_raw:', order_payments_raw.columns.tolist())
print('Columns in products_raw:', products_raw.columns.tolist())

print('\n--- Relationships ---')
print('1. orders_raw and order_items_raw: Can be joined using `order_id` (one order can have multiple items).')
print('2. orders_raw and order_payments_raw: Can be joined using `order_id` (one order can have multiple payments or payment types).')
print('3. order_items_raw and products_raw: Can be joined using `product_id` (each order item refers to a specific product).')
print('\nBy chaining these joins, you can link product information to orders and payments.')

Columns in orders_raw: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
Columns in order_items_raw: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']
Columns in order_payments_raw: ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']
Columns in products_raw: ['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

--- Relationships ---
1. orders_raw and order_items_raw: Can be joined using `order_id` (one order can have multiple items).
2. orders_raw and order_payments_raw: Can be joined using `order_id` (one order can have multiple payments or payment types).
3. order_items_raw and products_raw: Can be joined usin

In [33]:
products_raw.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [34]:
products_raw['product_category_name'].unique()

array(['perfumaria', 'artes', 'esporte_lazer', 'bebes',
       'utilidades_domesticas', 'instrumentos_musicais', 'cool_stuff',
       'moveis_decoracao', 'eletrodomesticos', 'brinquedos',
       'cama_mesa_banho', 'construcao_ferramentas_seguranca',
       'informatica_acessorios', 'beleza_saude', 'malas_acessorios',
       'ferramentas_jardim', 'moveis_escritorio', 'automotivo',
       'eletronicos', 'fashion_calcados', 'telefonia', 'papelaria',
       'fashion_bolsas_e_acessorios', 'pcs', 'casa_construcao',
       'relogios_presentes', 'construcao_ferramentas_construcao',
       'pet_shop', 'eletroportateis', 'agro_industria_e_comercio', nan,
       'moveis_sala', 'sinalizacao_e_seguranca', 'climatizacao',
       'consoles_games', 'livros_interesse_geral',
       'construcao_ferramentas_ferramentas',
       'fashion_underwear_e_moda_praia', 'fashion_roupa_masculina',
       'moveis_cozinha_area_de_servico_jantar_e_jardim',
       'industria_comercio_e_negocios', 'telefonia_fixa',
   

In [9]:
perfume_products = products_raw.loc[products_raw['product_category_name'] == 'perfumaria']
display(perfume_products.head())

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
16,6a2fb4dd53d2cdb88e0432f1284a004c,perfumaria,39.0,346.0,2.0,400.0,27.0,5.0,20.0
77,0d009643171aee696f4733340bc2fdd0,perfumaria,52.0,150.0,1.0,422.0,21.0,16.0,18.0
84,b1eae565a61935e0011ee7682fef9dc9,perfumaria,49.0,460.0,2.0,267.0,17.0,13.0,17.0
120,8da90b37f0fb171b4877c124f965b1f6,perfumaria,56.0,733.0,3.0,377.0,18.0,13.0,15.0


In [52]:
perfume_products.size

7812

In [10]:
beleza_saude_products = products_raw.loc[products_raw['product_category_name'] == 'beleza_saude']

In [54]:
beleza_saude_products.size

21996

In [13]:
beauty_items_raw = pd.concat([beleza_saude_products, perfume_products], axis=0)

In [56]:
beauty_items_raw.size

29808

In [14]:
raw_orders_beauty = pd.merge(beauty_items_raw, order_items_raw, how='inner', on='product_id')

In [58]:
print('Shape of beauty_items:', raw_orders_beauty.shape)
print('\nInfo about orders_with_items:')
raw_orders_beauty.info()
print('\nDescriptive statistics for raw_orders_beauty:')
display(raw_orders_beauty.describe())

Shape of beauty_items: (13089, 15)

Info about orders_with_items:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13089 entries, 0 to 13088
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  13089 non-null  object 
 1   product_category_name       13089 non-null  object 
 2   product_name_lenght         13089 non-null  float64
 3   product_description_lenght  13089 non-null  float64
 4   product_photos_qty          13089 non-null  float64
 5   product_weight_g            13089 non-null  float64
 6   product_length_cm           13089 non-null  float64
 7   product_height_cm           13089 non-null  float64
 8   product_width_cm            13089 non-null  float64
 9   order_id                    13089 non-null  object 
 10  order_item_id               13089 non-null  int64  
 11  seller_id                   13089 non-null  object 
 12  shipping_limit_date   

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_item_id,price,freight_value
count,13089.000000,13089.000000,13089.000000,13089.000000,13089.000000,13089.000000,13089.000000,13089.000000,13089.000000,13089.000000
mean,47.881504,1008.841164,1.768432,900.230957,22.734051,13.735579,17.803346,1.135763,126.656445,18.090043
std,10.062812,772.048729,1.356387,2520.834638,8.033744,8.835601,6.690171,0.654456,162.936370,15.416980
min,12.000000,47.000000,1.000000,50.000000,11.000000,2.000000,7.000000,1.000000,1.200000,0.000000
25%,42.000000,492.000000,1.000000,200.000000,18.000000,10.000000,15.000000,1.000000,44.890000,11.880000
50%,50.000000,791.000000,1.000000,400.000000,20.000000,12.000000,16.000000,1.000000,79.990000,15.450000
75%,56.000000,1257.000000,2.000000,755.000000,25.000000,16.000000,20.000000,1.000000,138.900000,19.120000
max,76.000000,3963.000000,14.000000,30000.000000,98.000000,105.000000,105.000000,21.000000,3124.000000,338.300000


In [15]:
beauty_products_relevant_columns = raw_orders_beauty.drop(['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm',
                                                           'product_height_cm', 'product_width_cm'], axis=1)

In [60]:
display(beauty_products_relevant_columns.describe())

,order_item_id,price,freight_value
count,13089.000000,13089.000000,13089.000000
mean,1.135763,126.656445,18.090043
std,0.654456,162.936370,15.416980
min,1.000000,1.200000,0.000000
25%,1.000000,44.890000,11.880000
50%,1.000000,79.990000,15.450000
75%,1.000000,138.900000,19.120000
max,21.000000,3124.000000,338.300000


In [64]:
beauty_products_relevant_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13089 entries, 0 to 13088
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_id             13089 non-null  object 
 1   product_category_name  13089 non-null  object 
 2   order_id               13089 non-null  object 
 3   order_item_id          13089 non-null  int64  
 4   seller_id              13089 non-null  object 
 5   shipping_limit_date    13089 non-null  object 
 6   price                  13089 non-null  float64
 7   freight_value          13089 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 818.2+ KB


###Obtaining now sellers information based on order items to finally get to geolocation

In [16]:
sellers_raw = pd.read_csv('/content/TCC_beauty/tcc-beleza/data/raw/models/olist_sellers_dataset.csv')

In [17]:
geolocation_raw = pd.read_csv('/content/TCC_beauty/tcc-beleza/data/raw/models/olist_geolocation_dataset.csv')

In [71]:
geolocation_raw.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [73]:
geolocation_raw.size

5000815

In [18]:
geolocation_raw_cleaned = geolocation_raw.drop_duplicates()

In [79]:
geolocation_raw_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 738332 entries, 0 to 1000161
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   geolocation_zip_code_prefix  738332 non-null  int64  
 1   geolocation_lat              738332 non-null  float64
 2   geolocation_lng              738332 non-null  float64
 3   geolocation_city             738332 non-null  object 
 4   geolocation_state            738332 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 33.8+ MB


In [19]:
geolocation_raw_cleaned = geolocation_raw_cleaned.rename(columns={'geolocation_zip_code_prefix': 'seller_zip_code_prefix'})

In [82]:
geolocation_raw_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 738332 entries, 0 to 1000161
Data columns (total 5 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   seller_zip_code_prefix  738332 non-null  int64  
 1   geolocation_lat         738332 non-null  float64
 2   geolocation_lng         738332 non-null  float64
 3   geolocation_city        738332 non-null  object 
 4   geolocation_state       738332 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 33.8+ MB


In [62]:
sellers_raw.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [24]:
beauty_products_sellers = pd.merge(beauty_products_relevant_columns, sellers_raw, how='inner', on='seller_id')

In [21]:
beauty_products_sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13089 entries, 0 to 13088
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_id              13089 non-null  object 
 1   product_category_name   13089 non-null  object 
 2   order_id                13089 non-null  object 
 3   order_item_id           13089 non-null  int64  
 4   seller_id               13089 non-null  object 
 5   shipping_limit_date     13089 non-null  object 
 6   price                   13089 non-null  float64
 7   freight_value           13089 non-null  float64
 8   seller_zip_code_prefix  13089 non-null  int64  
 9   seller_city             13089 non-null  object 
 10  seller_state            13089 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.1+ MB


In [29]:
# Deduplicate geolocation_raw_cleaned on 'seller_zip_code_prefix' to avoid row multiplication
geolocation_unique_zip = geolocation_raw_cleaned.drop_duplicates(subset=['seller_zip_code_prefix'])

beauty_by_sellers_geo = pd.merge(beauty_products_sellers, geolocation_unique_zip, how='left', on='seller_zip_code_prefix')

print(f'Length of beauty_products_sellers: {len(beauty_products_sellers)}')
print(f'Length of beauty_by_sellers_geo after deduplication and left merge: {len(beauty_by_sellers_geo)}')

display(beauty_by_sellers_geo.head())

Length of beauty_products_sellers: 13089
Length of beauty_by_sellers_geo after deduplication and left merge: 13089


,product_id,product_category_name,order_id,order_item_id,seller_id,shipping_limit_date,price,freight_value,seller_zip_code_prefix,seller_city,seller_state,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,e3e020af31d4d89d2602272b315c3f6e,beleza_saude,a41753c6a1d8accb89732e36243432d7,1,94144541854e298c2d976cb893b81343,2017-05-18 11:05:13,29.9,15.79,94451,viamao,RS,-30.159469,-50.832839,viamao,RS
1,c5d8079278e912d7e3b6beb48ecb56e8,beleza_saude,6a1594b5f5cfc5bac6dcdc3f48f22b5e,1,abcd2cb37d46c2c8fb1bf071c859fc5b,2018-03-12 18:30:16,95.9,27.22,78020,cuiaba,MT,-15.602979,-56.102960,cuiaba,MT
2,c5d8079278e912d7e3b6beb48ecb56e8,beleza_saude,f6fbf7907913892ffc12ada3bff286ba,1,abcd2cb37d46c2c8fb1bf071c859fc5b,2017-08-04 19:03:15,89.9,38.18,78020,cuiaba,MT,-15.602979,-56.102960,cuiaba,MT
3,36555a2f528d7b2a255c504191445d39,beleza_saude,a63144d37a00c28ef382668a5d5ece8b,1,851773c885feb5e2da3b4f82bc2b17ce,2018-02-14 14:31:45,29.9,16.11,18683,lencois paulista,SP,-22.602586,-48.812471,lencois paulista,SP
4,e586ebb6022265ae1eea38f46ffe3ead,beleza_saude,3ea7135b7064169abacfc10aa72f2e3a,1,01fdefa7697d26ad920e9e0346d4bd1b,2018-08-14 13:35:14,64.9,19.64,86050,londrina,PR,-23.331044,-51.177434,londrina,PR


In [30]:
display(beauty_by_sellers_geo.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13089 entries, 0 to 13088
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_id              13089 non-null  object 
 1   product_category_name   13089 non-null  object 
 2   order_id                13089 non-null  object 
 3   order_item_id           13089 non-null  int64  
 4   seller_id               13089 non-null  object 
 5   shipping_limit_date     13089 non-null  object 
 6   price                   13089 non-null  float64
 7   freight_value           13089 non-null  float64
 8   seller_zip_code_prefix  13089 non-null  int64  
 9   seller_city             13089 non-null  object 
 10  seller_state            13089 non-null  object 
 11  geolocation_lat         13089 non-null  float64
 12  geolocation_lng         13089 non-null  float64
 13  geolocation_city        13089 non-null  object 
 14  geolocation_state       13089 non-null

None

In [31]:
beauty_by_sellers_geo.to_csv('/content/TCC_beauty/tcc-beleza/notebooks/preproccessed_data_beauty.csv', index=False)

In [32]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    tcc-beleza/notebooks/olist_order_reviews_dataset.csv

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	tcc-beleza/data/raw/olist_order_reviews_dataset.csv
	tcc-beleza/notebooks/preproccessed_data_beauty.csv

no changes added to commit (use "git add" and/or "git commit -a")


In [33]:
%pwd

'/content/TCC_beauty'

UsageError: Line magic function `%whereami` not found.
